<h1>
<center>
Module 6: Random Forests
</center>
</h1>
<div class=h1_cell>

You will be working with the loan table again.

</div>

In [2]:
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
with open('/content/gdrive/My Drive/class_tables/loan_table_week4.csv', 'r') as f:
  loan_table = pd.read_csv(f)

In [0]:
!rm library_w19_week6.py

In [5]:
from google.colab import files
files.upload()

Saving library_w19_week6.py to library_w19_week6.py


{'library_w19_week6.py': b'import pandas as pd\nimport numpy as np\nfrom functools import reduce\nfrom types import SimpleNamespace\nimport random\n\ndef predictor_case(row, pred, target):\n\tcase_dict = {(0,0): \'true_negative\', (1,1): \'true_positive\', (0,1): \'false_negative\', (1,0): \'false_positive\'}\n\tactual = row[target]\n\tprediction = row[pred]\n\tcase = case_dict[(prediction, actual)]\n\treturn case\n\ndef accuracy(cases):\n\n    tp = 0\n    if \'true_positive\' in cases:\n        tp = cases[\'true_positive\']\n    tn = 0\n    if \'true_negative\' in cases:\n        tn = cases[\'true_negative\']\n    fp = 0\n    if \'false_positive\' in cases:\n        fp = cases[\'false_positive\']\n    fn = 0\n    if \'false_negative\' in cases:\n        fn = cases[\'false_negative\']\n\n    if(tp+tn+fp+fn) == 0:\n        return 0\n\n    return (tp + tn)/(tp+tn+fp+fn)\n\n#accuracy(p1_types)\n\ndef f1(cases):\n\n    #the heart of the matrix\n    #the heart of the matrix\n    tp = 0\n   

In [6]:
from library_w19_week6 import *

%who function

accuracy	 build_pred	 build_tree_iter	 compute_prediction	 compute_training	 f1	 find_best_splitter	 forest_builder	 forest_scores	 
generate_table	 gig	 gini	 informedness	 k_fold	 k_fold_random	 path_id	 predictor_case	 probabilities	 
produce_scores	 reorder_paths	 tree_predictor	 verify_unique	 vote_taker	 


In [7]:
pd.set_option('display.max_columns', None)
loan_table.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,lam_bin,lam_Low,lam_Average,lam_High,ch_bad,ch_good,ch_nan,apin_binned,apin_low,apin_average,apin_high,apin_nan,dep_0,dep_1,dep_2,dep_3+,dep_nan
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0,1,0,0,0,0
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0,0,128.000000,1,0,0,0,Low,1,0,0,0,1,0,low,1,0,0,0,0,1,0,0,0
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1,0,66.000000,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0,1,0,0,0,0
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1,0,120.000000,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0,1,0,0,0,0
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1,0,141.000000,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0,1,0,0,0,0


In [8]:
loan_table.columns.values

array(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'Loan_Status', 'no_lam', 'filled_lam', 'pa_Rural', 'pa_Semiurban',
       'pa_Urban', 'pa_nan', 'lam_bin', 'lam_Low', 'lam_Average',
       'lam_High', 'ch_bad', 'ch_good', 'ch_nan', 'apin_binned',
       'apin_low', 'apin_average', 'apin_high', 'apin_nan', 'dep_0',
       'dep_1', 'dep_2', 'dep_3+', 'dep_nan'], dtype=object)

<hr>
<h1>
1: Explore forest options (20)
</h1>
<p>
<div class=h1_cell>
<p>
Check out the results you get from forests of size 5, 11, 17.
<p>
First, define the columns to use. I do that for you below.
</div>

In [0]:
splitter_columns = [
        #Dependents
        'dep_0', 'dep_1', 'dep_2', 'dep_3+',
        #ApplicantIncome
       'apin_low', 'apin_high', 'apin_average',
        #Property_Area
        'pa_Rural', 'pa_Semiurban','pa_Urban',
        #LoanAmount
        'lam_Low', 'lam_Average', 'lam_High',
        #Credit_History
        'ch_bad', 'ch_good']

<hr>
<h2>
Set seeds so get consistent results
</h2>
<p>
<div class=h1_cell>
<p>

</div>

In [0]:
import numpy as np
import random

rng = np.random.RandomState(42)  #Will pass as arg to pandas sample method
random.seed(2000)

In [11]:
forest1 = forest_builder(loan_table, splitter_columns, 'Loan_Status', hypers={'total-trees': 5, 'random-state': rng})
len(forest1)

5

In [12]:
forest_scores(loan_table, forest1, 'Loan_Status')


[0.8094462540716613, 0.8764519535374868, 0.4104956556082149]

In [13]:
forest2 = forest_builder(loan_table, splitter_columns, 'Loan_Status', hypers={'total-trees': 11, 'random-state': rng})
len(forest2)

11

In [14]:
forest_scores(loan_table, forest2, 'Loan_Status')


[0.7785016286644951, 0.8597938144329897, 0.3058599921011058]

In [15]:
forest3 = forest_builder(loan_table, splitter_columns, 'Loan_Status', hypers={'total-trees': 17,'random-state': rng})
len(forest3)

17

In [16]:
forest_scores(loan_table, forest3, 'Loan_Status')


[0.8094462540716613, 0.8764519535374868, 0.4104956556082149]

<hr>
<h1>
2: Implement Out of Bag testing (80)
</h1>
<p>
<div class=h1_cell>
<p>
Last module we looked at the use of K-Folding as a means to test our trees. Random Forests give us an alternative by using out of bag testing. Using notes from the content notebook this week, find a way to do prediction using the oob idea. As reminder, the set union of all the oob lists in a forest make up the testing set. If there is a row in loan_table that is not in any oob list, that row should be omitted from the test set. Further, a tree only gets to vote on a specific row if that row is in the tree's oob list.
  <p>
  I am going to leave it to you to come up with an algorithm for doing oob testing. If you get totally stuck, I can supply hints. For grading I am looking to make sure you only use oob rows for testing and that each individual tree only votes on rows in its own oob list.
    <p>
      It is worthwhile solving this problem given something like it will likely be on next midterm.
</div>

In [0]:
def bag_test(forest):
  o_list = []
  for tree in forest:
    o_list += tree['oob']
  o_list = list(set(o_list))
  testing_table = loan_table.loc[o_list]
  testing_table = testing_table.reset_index()
  return testing_table

def bag_vote_taker(row, forest):
  votes = {0:0, 1:0}
  for tree in forest:
    # check to see if it is in oob
    if row['index'] in tree['oob']:
      prediction = tree_predictor(row, tree)
      votes[prediction] += 1
  winner = 1 if votes[1]>votes[0] else 0  #ties go to 0
  return winner

def bag_forest_scores(table, forest, target):
    scratch_table = pd.DataFrame(columns=['prediction', 'actual'])
    scratch_table['prediction'] = table.apply(lambda row: bag_vote_taker(row, forest), axis=1)
    scratch_table['actual'] = table[target]  # just copy the target column
    cases = scratch_table.apply(lambda row: predictor_case(row, pred='prediction', target='actual'), axis=1)
    vc = cases.value_counts()
    return [accuracy(vc), f1(vc), informedness(vc)]


<h2>Check your oob testing against my results</h2>

If you used the random seeds to build your trees, your results should be the same as mine. No randomness during oob testing.

In [18]:
#whole table from above: [0.8094462540716613, 0.8764519535374868, 0.4104956556082149]
testing_table_1 = bag_test(forest1)
bag_forest_scores(testing_table_1, forest1, 'Loan_Status')


[0.7887067395264117, 0.8595641646489104, 0.39501598819333417]

In [0]:
testing_table_2 = bag_test(forest2)


In [20]:
#from above: [0.7785016286644951, 0.8597938144329897, 0.3058599921011058]

bag_forest_scores(testing_table_2, forest2, 'Loan_Status')


[0.7586206896551724, 0.8457502623294858, 0.2730153560960946]

In [0]:
testing_table_3 = bag_test(forest3)

In [22]:
#from above: [0.8094462540716613, 0.8764519535374868, 0.4104956556082149]

bag_forest_scores(testing_table_3, forest3, 'Loan_Status')


[0.7768729641693811, 0.8568443051201673, 0.32052231437598744]

<h2>Not a lot of change</h2>

Using oob testing did not affect scores much. I think we would need to work with bigger tables, e.g., the 25K shelter table, to see a difference.

<hr>
<h1>Write it out</h1>
<div class=h1_cell>

Did not change table but we did define new functions. Add them to your library as `!rm library_w19_week6b.py`. I added the `b` to designate functions from assignment portion of module.
</div>